In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

model = ChatOpenAI(model="gpt-4o-mini")

model.invoke(
    [
        HumanMessage(content="Hi! My name is Dd."),
        AIMessage(content="Hello Dd! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Dd! How can I help you today, Dd?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 38, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-59880d39-8c77-49ca-abfe-cb2fee548123-0', usage_metadata={'input_tokens': 38, 'output_tokens': 17, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [4]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

In [5]:
# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

In [6]:
# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

In [7]:
# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [8]:
# Config into runnable
config = {"configurable": {"thread_id": "dd001"}}

In [9]:
# Invoke the application

query = "Hi! My name is Dd."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Dd! How can I assist you today?


In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [14]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):                                                       # The new typeddict
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "dd003"}}
query = "Hi! My name is Dd."
language = "Tagalog"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},                    # Includes the parameters
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Kamusta, Dd! Ano ang maitutulong ko sa iyo ngayon?
